In [218]:
'''
This cell will extract extra info about Segment and Volume from each ticker and save it into a csv file.
'''

import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.oceans14.com.br/acoes/ranking/liquidez"

headers = {
    'User-Agent': 'Nobody'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
verAcoes_section = soup.find('section', {'id': 'verAcoes'})

# Extracting the columns
tickers = []
segments = []
unrefined_volumes = []

# Find all rows in the table
rows_tickers = verAcoes_section.find_all('td', style="text-align:left")
rows_segment = verAcoes_section.find_all('td', class_="hidden-xs hidden-sm")

# Extracting tickers from the rows
for index, row in enumerate(rows_tickers):
    link = row.find('a')  # Find the <a> tag within the row
    if link:  # If <a> tag is found
        ticker = link.text  # Extract text (ticker symbol)
        tickers.append(ticker)  # Append the ticker to the tickers list
    
    # if index % 4 == 0:
    unrefined_volume = row.text
    unrefined_volumes.append(unrefined_volume)

for index, row in enumerate(rows_segment):
    if index % 2:
        segment = row.text
        segments.append(segment)


# Obtaining the values at specific indices (5, 10, 15...)
indices_specific = [i for i in range(4, len(unrefined_volumes), 5)]

# Retrieving the values from the "data" list using the specific indices
volumes = [unrefined_volumes[i] for i in indices_specific]

# Combine tickers and segments into a dictionary
data = {
    'Ticker': tickers,
    'Segment': segments,
    'Volume' : volumes
}


# Create a Pandas DataFrame from the dictionary
df = pd.DataFrame(data)
df['Segment'] = df['Segment'].str.upper()
df['Volume'] = df['Volume'].str.replace('.', '')
df = df.drop_duplicates()

# df.to_csv('tickers_segments.csv', index=False)

In [31]:
from pyspark.sql.functions import col, regexp_replace, sqrt, round
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType

# Create a SparkSession
spark = SparkSession.builder.appName("InvestmentAnalysis").getOrCreate()

def process_csv_to_dataframe(csv_file, dataframe_name):
    
    # Read the CSV file
    df = spark.read.csv(csv_file, header=True, sep=";")

    # Iterate through the remaining columns and apply transformations based on column names
    for col_name in df.columns[1:]:  # Start from the second column
        new_col_name = col_name.strip().replace(" / ", "_").replace(" ", "_").replace("/", "_").replace(".", "").strip()
        df = df.withColumnRenamed(col_name, new_col_name)

        if new_col_name in ["LIQUIDEZ_MEDIA_DIARIA", "VALOR_DE_MERCADO"]:
            df = df.withColumn(new_col_name, regexp_replace(col(new_col_name), "[.]", ""))
            df = df.withColumn(new_col_name, regexp_replace(col(new_col_name), "[,]", ".").cast(DoubleType()))    
        else:
            df = df.withColumn(new_col_name, regexp_replace(col(new_col_name), ",", ".").cast(DoubleType()))

    df = df.withColumn('VI', round(sqrt(22.5 * col('LPA') * col('VPA')), 2))
    df.createOrReplaceTempView(dataframe_name)

process_csv_to_dataframe("tickers.csv", "all_tickers")
# process_csv_to_dataframe("bancos.csv", "banks")
# process_csv_to_dataframe("energia_eletrica.csv", "eletric_energy")

spark.read.csv("tickers_segments.csv", header=True, sep=",").createOrReplaceTempView("tickers_segments_unrefined")

# Adding segment and volume on tickers view
spark.sql('''
CREATE OR REPLACE TEMP VIEW all_tickers_completed AS
          
SELECT a.*, b.SEGMENT, b.VOLUME
FROM all_tickers a
LEFT JOIN (
    SELECT TICKER, SEGMENT, CAST(VOLUME AS DOUBLE) AS VOLUME
    FROM tickers_segments_unrefined
) b
  ON a.TICKER = b.TICKER
''')

DataFrame[]

In [19]:
'''
TODO: check if there are better segments and tickers to add
'''

# # Test
# spark.sql('''
# SELECT PATRIMONIO_ATIVOS FROM all_tickers_completed WHERE PATRIMONIO_ATIVOS < 0
# ''').show(100, truncate=False)

'\nTODO: check if there are better segments and tickers to add\n'

In [20]:
# Data per segment
spark.sql('''
SELECT 
  SEGMENT, 
  ROUND(AVG(DY),2) AS DY, ROUND(AVG(ROE),2) AS ROE, 
  ROUND(AVG(P_VP),2) AS P_VP, ROUND(AVG(P_L),2) AS P_L, 
  ROUND(AVG(DIV_LIQ_PATRI),2) AS DIV_LIQ_PATRI,
  ROUND(AVG(PATRIMONIO_ATIVOS),2) AS PATRIMONIO_ATIVOS 
          
FROM all_tickers_completed
WHERE 1=1 
  AND DY > 0 AND P_L > 0 AND P_VP > 0 AND ROE > 0 AND PATRIMONIO_ATIVOS > 0
  AND SEGMENT IN ("QUÍMICOS DIVERSOS", "MINERAIS METÁLICOS", "SEGURADORAS", "ÁGUA E SANEAMENTO", "ENERGIA ELÉTRICA", "BANCOS", "MOTORES, COMPRESSORES E OUTROS", "PAPEL E CELULOSE", "PRODUTOS PARA CONSTRUÇÃO", "EXPLORAÇÃO, REFINO E DISTRIBUIÇÃO", "SIDERURGIA", "TELECOMUNICAÇÕES")
  GROUP BY SEGMENT
  ORDER BY DY DESC, ROE DESC
''').show(100, truncate=False)

+---------------------------------+-----+-----+----+-----+-------------+-----------------+
|SEGMENT                          |DY   |ROE  |P_VP|P_L  |DIV_LIQ_PATRI|PATRIMONIO_ATIVOS|
+---------------------------------+-----+-----+----+-----+-------------+-----------------+
|QUÍMICOS DIVERSOS                |16.91|22.45|2.2 |10.1 |-0.08        |0.59             |
|SIDERURGIA                       |12.05|19.4 |0.81|4.13 |0.14         |0.51             |
|MINERAIS METÁLICOS               |10.41|26.82|1.65|7.15 |0.11         |0.65             |
|EXPLORAÇÃO, REFINO E DISTRIBUIÇÃO|9.74 |20.62|1.53|13.03|0.59         |0.4              |
|SEGURADORAS                      |9.02 |41.7 |3.74|9.1  |-0.53        |0.58             |
|ÁGUA E SANEAMENTO                |7.28 |14.53|0.89|6.47 |0.49         |0.52             |
|BANCOS                           |6.0  |12.87|1.14|9.2  |NULL         |0.09             |
|ENERGIA ELÉTRICA                 |5.86 |15.4 |1.45|13.22|0.97         |0.33             |

In [32]:
#Query specific segment
spark.sql('''
SELECT TICKER, SEGMENT, PRECO, VI, ROUND(PRECO/VI,3) AS PRECO_VI_DIVISION, DY, ROE, P_VP, P_L, DIV_LIQ_PATRI, PATRIMONIO_ATIVOS, VOLUME
FROM all_tickers_completed
where 1=1 
  AND SEGMENT = "ENERGIA ELÉTRICA"           
  ORDER BY DY DESC, VOLUME DESC
''').show(100, truncate=False)

+------+----------------+-----+-----+-----------------+-----+------+----+-----+-------------+-----------------+-------------+
|TICKER|SEGMENT         |PRECO|VI   |PRECO_VI_DIVISION|DY   |ROE   |P_VP|P_L  |DIV_LIQ_PATRI|PATRIMONIO_ATIVOS|VOLUME       |
+------+----------------+-----+-----+-----------------+-----+------+----+-----+-------------+-----------------+-------------+
|AURE3 |ENERGIA ELÉTRICA|13.89|25.25|0.55             |11.13|14.52 |0.97|6.65 |0.09         |0.45             |1.217467468E9|
|TAEE3 |ENERGIA ELÉTRICA|11.69|12.83|0.911            |9.95 |17.46 |1.75|10.03|1.21         |0.36             |3.027026E7   |
|TAEE11|ENERGIA ELÉTRICA|35.14|38.49|0.913            |9.92 |17.46 |1.75|10.05|1.21         |0.36             |1.520974579E9|
|TAEE4 |ENERGIA ELÉTRICA|11.73|12.83|0.914            |9.87 |17.46 |1.76|10.11|1.21         |0.36             |5.8508724E7  |
|REDE3 |ENERGIA ELÉTRICA|5.73 |5.01 |1.144            |9.4  |34.34 |3.19|9.28 |2.57         |0.14             |213852.

In [33]:
# Possible Wallet
spark.sql('''
SELECT TICKER, SEGMENT, PRECO, VI, ROUND(PRECO/VI,3) AS PRECO_VI_DIVISION, DY, ROE, P_VP, P_L, DIV_LIQ_PATRI, VOLUME
FROM all_tickers_completed
where 1=1 
  AND TICKER IN ("PETR3", "CPLE6", "TAEE11", "CMIG4", "KLBN11", "SUZB3", "BBAS3", "ITUB4", "SANB11", "UNIP6", "ITSA4", "LEVE3", "CSNA4", "GGBR4", "VALE3")
  ORDER BY SEGMENT, DY DESC
''').show(100, truncate=False)

+------+---------------------------------+-----+------+-----------------+-----+-----+----+-----+-------------+---------------+
|TICKER|SEGMENT                          |PRECO|VI    |PRECO_VI_DIVISION|DY   |ROE  |P_VP|P_L  |DIV_LIQ_PATRI|VOLUME         |
+------+---------------------------------+-----+------+-----------------+-----+-----+----+-----+-------------+---------------+
|LEVE3 |AUTOMÓVEIS E MOTOCICLETAS        |30.5 |37.11 |0.822            |11.87|49.55|3.3 |6.67 |0.13         |8.81483881E8   |
|BBAS3 |BANCOS                           |50.45|120.44|0.419            |9.45 |20.01|0.86|4.28 |NULL         |1.0348598464E10|
|SANB11|BANCOS                           |27.82|41.16 |0.676            |6.12 |8.25 |0.9 |10.92|NULL         |1.614989311E9  |
|ITUB4 |BANCOS                           |28.19|36.35 |0.776            |5.49 |17.65|1.49|8.47 |NULL         |1.4848193214E10|
|TAEE11|ENERGIA ELÉTRICA                 |35.14|38.49 |0.913            |9.92 |17.46|1.75|10.05|1.21         |1